## Import library

In [28]:
import torch
from torch import nn
import tqdm

In this model, due to the limited resources I had, I wasn’t able to provide the model with all the necessary data, nor could I train it for the required number of epochs. But since the structure is correct, it’s easy to run this process anyway.

In [29]:
text='''I am Harry Potter. I am a young boy who lives with my aunt and uncle, the Dursleys, in a small house on Privet Drive. I am curious about the world and I often wonder why I feel different from others. One day, I receive a letter from Hogwarts School of Witchcraft and Wizardry. I am surprised and excited because I learn that I am a wizard. I have never seen magic before, and I am eager to discover all the wonders that await me. I pack my things and travel to Hogwarts, feeling nervous and thrilled at the same time. I meet many students on the train and I make friends with Ron Weasley and Hermione Granger. I am happy to have friends who understand me and share my excitement. We explore the castle together and discover hidden rooms, secret passages, and magical creatures. I see dragons, hippogriffs, house-elves, unicorns, and other creatures I have never imagined. I learn many spells and practice them every day. I try simple charms, defensive spells, and more powerful magic. I am careful when using magic because I want to do things correctly and safely. I attend many classes at Hogwarts, including Potions, Defense Against the Dark Arts, Transfiguration, Herbology, and Flying Lessons. I play Quidditch on a broomstick and practice teamwork with my friends. I learn strategy, courage, and cooperation. I face many challenges and adventures. I discover the secret of the Philosopher’s Stone and protect it from those who want to steal it. I fight trolls, solve puzzles, and confront dark wizards. I learn that courage, loyalty, and friendship are more important than any spell. I help my friends when they are in danger and I protect those who need help. I practice magic every day and read books to improve my knowledge. I observe magical creatures and learn how to care for them. I explore the castle and the grounds, discovering new places and hidden secrets. I meet ghosts, magical plants, and enchanted objects. I try to understand their powers and how to use them responsibly. I grow stronger and wiser with every year at Hogwarts. I learn about the history of magic and the ongoing fight between good and evil. I discover that love and loyalty are powerful forms of magic that cannot be defeated. I practice spells, study lessons, and learn life lessons. I make mistakes, but I try to learn from them. I act with courage even when situations are dangerous or uncertain. I share my knowledge with friends and help them overcome challenges. I remember the advice of my teachers and follow guidance when necessary. I explore secret corridors, hidden rooms, and mysterious passages. I encounter magical creatures that teach me new things about the world. I care for my friends and help them when they are in trouble. I feel proud when we succeed together and celebrate small victories. I understand that honesty, bravery, and kindness are stronger than any magic spell. I practice patience and persistence, even when learning is difficult. I know that helping others is more important than personal gain. I continue to explore Hogwarts and the magical world. I meet new students, learn new spells, and discover magical plants and objects. I face challenges that test my courage, intelligence, and compassion. I try to be responsible and protect others whenever possible. I grow more confident and wise each year. I think before I act and trust my instincts. I understand that mistakes are lessons and challenges are opportunities. I feel proud of my achievements and grateful for my friends and teachers. I continue to practice magic, learn new spells, and explore the magical world. I believe that even ordinary people can do extraordinary things if they believe in themselves and support others. I am Harry Potter, a young wizard learning about magic, friendship, courage, love, and responsibility. I am ready to face new adventures, help my friends, and protect the magical world. I am determined to be brave, kind, and wise. I am learning, exploring, and growing every day. I discover new things about magic, about people, and about myself. I face dangers, solve mysteries, and meet magical creatures. I practice spells, play Quidditch, and attend lessons. I learn teamwork, responsibility, and courage. I help friends, protect Hogwarts, and try to do what is right. I continue to discover secrets of the castle, learn new magic, and improve my skills. I face dark forces and learn how to fight them. I understand that courage, hope, and friendship are more powerful than fear. I act with kindness, loyalty, and bravery in every situation. I continue to grow as a wizard and as a person. I explore, learn, and practice magic every day. I believe in myself and in the power of my friends. I know that together we can overcome obstacles and achieve extraordinary things. I am ready for new challenges, new lessons, and new adventures. I continue to write my story at Hogwarts, learning from each day, and sharing experiences with friends. I discover the importance of love, courage, and perseverance. I help others when they need me and I protect those who cannot protect themselves. I practice magic responsibly, learn from mistakes, and grow stronger every day. I am Harry Potter, a wizard who is learning about life, magic, friendship, and courage. I am ready to face the world with bravery, wisdom, and kindness. I continue to learn, explore, and grow, discovering the magical world and my place in it.'''

In [30]:
#all words
splited_word=text.split()
vocab=set(list(splited_word))
hidden_size=20
#get index of word
word2idx={word:i for i,word in enumerate(vocab)}
#get word of index
idx2word={i:word for i,word in enumerate(vocab)}

## Model

In [31]:
class lstm(nn.Module):
    def __init__(self,vocab_size,hidden_size,embed_dim,device):
        super().__init__()

        self.hidden_size=hidden_size

        #embeding words


        self.embeding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_dim)


        #parameters of forget gate
        self.forgetw=nn.Parameter(torch.empty(hidden_size,embed_dim))
        self.forgetu=nn.Parameter(torch.empty(hidden_size,hidden_size))
        self.forgetb=nn.Parameter(torch.zeros(hidden_size))

        #patameters of input gate

        self.inputw=nn.Parameter(torch.empty(hidden_size,embed_dim))
        self.inputu=nn.Parameter(torch.empty(hidden_size,hidden_size))
        self.inputb=nn.Parameter(torch.zeros(hidden_size))

        #parameters of output gate

        self.outputw=nn.Parameter(torch.empty(hidden_size,embed_dim))
        self.outputu=nn.Parameter(torch.empty(hidden_size,hidden_size))
        self.outputb=nn.Parameter(torch.zeros(hidden_size))

        #paramters of cell state

        self.cellstatew=nn.Parameter(torch.empty(hidden_size,embed_dim))
        self.cellstateu=nn.Parameter(torch.empty(hidden_size,hidden_size))
        self.cellstateb=nn.Parameter(torch.zeros(hidden_size))



        self.ht=torch.ones(self.hidden_size)
        self.c=torch.ones(self.hidden_size)

        self.sigmoid=nn.Sigmoid()

        self.tanh=nn.Tanh()


        self.decoder=nn.Linear(hidden_size,vocab_size)

        self.device=device

        #initialization all parameters

        self.reset_parameter()







    def reset_parameter(self):

        # initialization all weight with xavier
        for param in self.parameters():
            if param.dim() > 1:
                nn.init.xavier_uniform_(param)

            #initialization all bias with zeros
            else :
                nn.init.zeros_(param)

        #initialization fort biyas
        self.forgetb.data.fill_(1.0)



    def forward(self,idx,ht,c):

        #index
        idx=torch.tensor(idx)
        # get embeded of words

        embeded_word=self.embeding(idx)




        f=self.sigmoid( self.forgetw @ embeded_word + self.forgetu@ ht  + self.forgetb )

        i=self.sigmoid( self.inputw @ embeded_word + self.inputu @ ht + self.inputb )

        o=self.sigmoid( self.outputw @ embeded_word + self.outputu @ ht + self.outputb)

        c_=self.tanh( self.cellstatew @ embeded_word + self.cellstateu @ ht +self.cellstateb)

        c=f * c + i * c_

        ht = o * self.tanh(c)

        output=self.decoder(ht)









        return ht,c,output


In [32]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model=lstm(vocab_size=len(vocab),hidden_size=hidden_size,embed_dim=100,device=device).to(device)

optim=torch.optim.Adam(lr=1e-4,params=model.parameters())

loss_fn=nn.CrossEntropyLoss()


## Train

In [37]:
train_loss=[]
model.train()

ht=torch.zeros((hidden_size))
c=torch.zeros((hidden_size))


for i in tqdm.trange(20):
    for i in range(len(splited_word)-1):
        optim.zero_grad()
        # true word
        #get true word of before step time so it is tacher forecing
        input_word=splited_word[i]
        input_idx=(torch.tensor(word2idx[input_word]).long()).to(device)


        ht,c,decoder=model.forward(idx=input_idx,ht=ht,c=c)


        test_word=splited_word[i+1]
        test_idx=(torch.tensor(word2idx[test_word]).long()).to(device)


        loss=loss_fn(decoder,test_idx)

        train_loss.append(loss)


        loss.backward()

        optim.step()


        ht=ht.detach()
        c=c.detach()
















  0%|          | 0/20 [00:00<?, ?it/s]/tmp/ipython-input-2679858664.py:79: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  idx=torch.tensor(idx)
100%|██████████| 20/20 [01:03<00:00,  3.19s/it]


# Generate text

In [38]:
len_text=10
first_word=splited_word[0]
idx=(torch.tensor(word2idx[first_word]).long())
for i in range(len_text):
  ht,c,decoder=model.forward(idx=idx,ht=ht,c=c)
  idx=torch.argmax(decoder)
  print(idx2word[idx.item()])


am
careful
I
am
am
careful
I
am
am
careful


/tmp/ipython-input-2679858664.py:79: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  idx=torch.tensor(idx)
